In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

from torch_geometric.utils import from_networkx, to_networkx
import networkx as nx

import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint, Callback, TQDMProgressBar
from pytorch_lightning.loggers import WandbLogger



from typing import Callable, Optional

In [3]:
import sys
sys.path.insert(0,'..')

In [4]:
from utils.heterophilic import get_dataset, get_fixed_splits, WebKB, Actor
from project.sheaf_datamodule import SheafDataset_ForGraphs, SheafDataModule_ForGraphs
from project.sheaf_learner import coboundary_learner_on_graph_signal
from project.sheaf_diffusion import sheaf_diffusion
from project.sheaf_simultaneous_sparse import sheaf_gradient_flow
from project.sheaf_simultaneous_potential import sheaf_gradient_flow_potential
from project.functor_stacked_energy_accelerate_edges import sheaf_gradient_flow_functor

In [5]:
import os

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

from torch_geometric.utils import from_networkx, to_networkx


import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint, Callback, TQDMProgressBar
from pytorch_lightning.loggers import WandbLogger



from typing import Callable, Optional


import sys
sys.path.insert(0,'..')
from utils.heterophilic import get_dataset, get_fixed_splits, WebKB, Actor
from project.sheaf_datamodule import SheafDataset_ForGraphs, SheafDataModule_ForGraphs
from project.sheaf_learner import coboundary_learner_on_graph_signal

from project.sheaf_edgewise_functor_version2  import sheaf_gradient_flow_functor

import wandb

wandb.login()

dataset = get_dataset('film')
data_source = dataset[0]




sweep_config = {
    'method': 'random'
    }


metric = {
    'name': 'val_acc',
    'goal': 'maximize'
    }

sweep_config['metric'] = metric

parameters_dict = {
    'dv': {
        'distribution': 'categorical',
        'values': [2,3,4,5]
    },
    'de': {
        'distribution': 'categorical',
        'values': [2,3,4,5]
    },
    'input_dropout': {
        'distribution': 'categorical',
        'values':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        #'values':[0.0]
    },
    'dropout': {
        'distribution': 'categorical',
        #'values':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        'values':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6,0.7, 0.8, 0.9]
    },
    'fold': {
        'distribution': 'categorical',
        #'values':[0,1,2,3,4,5,6,7,8,9],
        'values':[0,1,2,3,4,5,6,7,8,9]
    },
    'layers':{
        'distribution': 'categorical',
        'values': [1,2,3,4]
    },
    'NVertEnergy':{
        'distribution': 'categorical',
        'values': [0, 1, 2, 3]
    },
    'NEdgeEnergy':{
        'distribution': 'categorical',
        'values': [1, 2, 3]
    },
    'channels':{
        'distribution': 'categorical',
        'values': [2,4,8,16,32]
    },
    'batch_size':{
        'distribution': 'categorical',
        'values': [1]
    },
    'first_hidden':{
        'distribution':'categorical',
        'values':[3000, 1500,1000,800,600, 700, 500,400,250, 100, 50]
    },
    'second_hidden':{
        'distribution':'categorical',
        'values':[3000, 1500,1000,800, 600,700,500, 400, 250,100, 50]
    },
    'use_act':{
        'distribution':'categorical',
        'values':[True, False]
    },
    'free_potential':{
        'distribution':'categorical',
        'values':[1,2]
    },
    'learning_rate':{
        'distribution':'categorical',
        'values':[1e-3, 5e-4, 5e-3]
    },
    'weight_decay':{
        'distribution':'log_uniform',
        'min':-9.2,
        'max':-6.8
    },
    'nonlinearoutput':{
        'distribution':'categorical',
        'values':[True, False]
    },
    'add_adjacency':{
        'distribution':'categorical',
        'values':[True, False]
    },
    'add_stalk_mixing':{
        'distribution':'categorical',
        'values':[True, False]
    },
    'add_channel_mixing':{
        'distribution':'categorical',
        'values':[True, False]
    },
    'apply_spectralnorm':{
        'distribution':'categorical',
        'values':[True, False]
    },
    'apply_symmetrize':{
        'distribution':'categorical',
        'values':[True, False]
    },
    'vertex_potential_type':{
        'distribution':'categorical',
        'values':[0, 1]
    }

}

sweep_config['parameters'] = parameters_dict

import pprint
pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config, project="Sweep_functor_onlyxflow_stackenergy_Jan_13_chameleon")

wandb: Currently logged in as: saepark. Use `wandb login --relogin` to force relogin


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_acc'},
 'parameters': {'NEdgeEnergy': {'distribution': 'categorical',
                                'values': [1, 2, 3]},
                'NVertEnergy': {'distribution': 'categorical',
                                'values': [0, 1, 2, 3]},
                'add_adjacency': {'distribution': 'categorical',
                                  'values': [True, False]},
                'add_channel_mixing': {'distribution': 'categorical',
                                       'values': [True, False]},
                'add_stalk_mixing': {'distribution': 'categorical',
                                     'values': [True, False]},
                'apply_spectralnorm': {'distribution': 'categorical',
                                       'values': [True, False]},
                'apply_symmetrize': {'distribution': 'categorical',
                                     'values': [True, False]},
                'batch_size': {'d

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. weight_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: f42yrtrl
Sweep URL: https://wandb.ai/saepark/Sweep_functor_onlyxflow_stackenergy_Jan_13_chameleon/sweeps/f42yrtrl


In [6]:
dataset = get_dataset('chameleon')
data = dataset[0]
data = get_fixed_splits(data, 'chameleon', 0)

In [10]:
data

Data(x=[2277, 2325], edge_index=[2, 62742], y=[2277], train_mask=[2277], val_mask=[2277], test_mask=[2277])

In [8]:
min(data.y)

tensor(0)

In [9]:
max(data.y)

tensor(4)

In [6]:
graph = to_networkx(data)

In [7]:
graph = graph.to_undirected()

In [8]:
len(graph)

2277

In [24]:
Nv = len(graph)
Ne = graph.number_of_edges()
#dv = 3
#de = 2
#final_d = dv + 2
#hidden_channels = 4
#edge = torch.randn(Ne, 2*final_d*hidden_channels + 2*dv*de)

In [25]:
Ne

31371

edge[0]

In [26]:
from_vertex_outgoing_edges = [ [] for _ in range(Nv) ]
from_vertex_incoming_edges = [ [] for _ in range(Nv) ]
from_vertex_outgoing_vertex = [ [] for _ in range(Nv) ]
from_vertex_incoming_vertex = [ [] for _ in range(Nv) ]
for i, (v1, v2) in enumerate(graph.edges):
    from_vertex_outgoing_edges[v1].append(i)
    from_vertex_incoming_edges[v2].append(i)
    from_vertex_outgoing_vertex[v1].append(v2)
    from_vertex_incoming_vertex[v2].append(v1)

In [27]:
degree_list = []
for i in range(Nv):
    degree_list.append(graph.degree(i))

In [28]:
degree_list = torch.tensor(degree_list)

In [29]:
for i in range(Nv):
    from_vertex_outgoing_edges[i] = torch.tensor(from_vertex_outgoing_edges[i])
    from_vertex_outgoing_vertex[i] = torch.tensor(from_vertex_outgoing_vertex[i])
    from_vertex_incoming_edges[i] = torch.tensor(from_vertex_incoming_edges[i])
    from_vertex_incoming_vertex[i] = torch.tensor(from_vertex_incoming_vertex[i])

In [30]:
mask = {'train_mask':data.train_mask, 
        'val_mask':data.val_mask, 
        'test_mask':data.test_mask
       }

In [31]:
data

Data(x=[2277, 2325], edge_index=[2, 62742], y=[2277], train_mask=[2277], val_mask=[2277], test_mask=[2277])

In [32]:
max(data.y)

tensor(4)

In [33]:
min(data.y)

tensor(0)

In [34]:
#    def __init__(self, graph, Nv, dv, Ne, de, adjacency, max_deg, layers, input_dim, output_dim, channels, left_weights, right_weights, potential, mask, use_act, augmented, add_lp, add_hp, dropout, input_dropout, free_potential, first_hidden, second_hidden, learning_rate = 0.01):

#input_dim = 2089
input_dim = 2325
output_dim = 5

dv = 3
de = 3
layers = 3
channels = 8
left_weights = True
right_weights = True
 
dropout     = 0.3
input_dropout = 0.
free_potential = 2

first_hidden =1000
second_hidden=200

NVertEnergy = 0
NEdgeEnergy = 1

learning_rate = 1e-3
weight_decay  = 5e-4

nonlinearoutput = False
add_adjacency = True
add_stalk_mixing = False
add_channel_mixing = True
activation = False

apply_spectralnorm = False
apply_symmetrize = False
vertex_potential_type = 1
#Towards final, 
model_diffusion = sheaf_gradient_flow_functor(graph,from_vertex_outgoing_edges, from_vertex_incoming_edges, from_vertex_outgoing_vertex, from_vertex_incoming_vertex, degree_list, graph.number_of_nodes(), dv, graph.number_of_edges(), de, layers,input_dim,output_dim, channels, left_weights, right_weights, 'radial', mask, activation,True,True,True, dropout,input_dropout,free_potential,first_hidden,second_hidden,NVertEnergy, NEdgeEnergy, nonlinearoutput,add_adjacency, add_stalk_mixing, add_channel_mixing, apply_spectralnorm, apply_symmetrize, vertex_potential_type,learning_rate, weight_decay)


#New_Theory
#YES ACTIVATION
#model_diffusion = sheaf_gradient_flow_functor(graph, graph.number_of_nodes(), dv, graph.number_of_edges(), de, layers,input_dim,output_dim, channels, left_weights, right_weights, 'radial', mask, True,True,True,True, dropout,input_dropout,free_potential,first_hidden,second_hidden,learning_rate, weight_decay)

#NO ACTIVATION
#model_diffusion = sheaf_gradient_flow_functor(graph, graph.number_of_nodes(), dv, graph.number_of_edges(), de, layers,input_dim,output_dim, channels, left_weights, right_weights, 'radial', mask, False,True,True,True, dropout,input_dropout,free_potential,first_hidden,second_hidden,learning_rate, weight_decay)

#STACK ENERGY, NO ACTIVATION
#model_diffusion = sheaf_gradient_flow_functor(graph, graph.number_of_nodes(), dv, graph.number_of_edges(), de, layers,input_dim,output_dim, channels, left_weights, right_weights, 'radial', mask, False,True,True,True, dropout,input_dropout,free_potential,first_hidden,second_hidden,NVertEnergy, NEdgeEnergy, learning_rate, weight_decay)

#STACK ENERGY, YES ACTIVATION
#model_diffusion = sheaf_gradient_flow_functor(graph, graph.number_of_nodes(), dv, graph.number_of_edges(), de, layers,input_dim,output_dim, channels, left_weights, right_weights, 'radial', mask, True,True,True,True, dropout,input_dropout,free_potential,first_hidden,second_hidden,NVertEnergy, NEdgeEnergy, learning_rate, weight_decay)

/root/projects/sheaf-learn-project/notebooks/../project/functor_stacked_energy_accelerate_edges.py:168: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph)
<class 'networkx.utils.decorators.argmap'> compilation 4:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [35]:
#model_diffusion

In [36]:
#import wandb
#wandb.finish()

In [37]:
#wandb_logger_diffusion = WandbLogger(project="functor_final_squirrel_0")
wandb_logger_diffusion = WandbLogger(project="functor_accelerate_chameleon_0")

wandb: Currently logged in as: saepark. Use `wandb login --relogin` to force relogin


In [38]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.000, patience=200, verbose=False)
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="/root/projects/temp_checkpoints/chameleon_0/edge_accelerate",
    filename="memory-functoredge-vertex-edge-Jan02-{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}",
    save_top_k=20,
    mode="min",
)
class PrintCallbacks(Callback):
    def on_init_start(self, trainer):
        print("Starting to init trainer!")

    def on_init_end(self, trainer):
        print("Trainer is init now")

    def on_train_end(self, trainer, pl_module):
        print("Training ended")


In [39]:
sheaf_dm = SheafDataModule_ForGraphs(data, 1 , 1)

In [40]:
trainer = Trainer(accelerator='gpu',devices=[0],callbacks=[TQDMProgressBar(refresh_rate=10),PrintCallbacks(),early_stop_callback,checkpoint_callback],logger=wandb_logger_diffusion,auto_lr_find=True)

Starting to init trainer!


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Trainer is init now


lr_finder = trainer.tuner.lr_find(model_diffusion, sheaf_dm)

## model_diffusion

fig = lr_finder.plot(suggest=True)
fig.show()

In [41]:
#new_lr = lr_finder.suggestion()
#model_diffusion.hparams.learning_rate = 5e-3
#model_diffusion.learning_rate = 5e-3
#model_diffusion.hparams.learning_rate = new_lr
#model_diffusion.learning_rate = new_lr
#print(new_lr)

In [ ]:
trainer.fit(model_diffusion, sheaf_dm)

/root/miniconda3/envs/sheaflearn/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:89: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
/root/miniconda3/envs/sheaflearn/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:229: LightningDeprecationWarning: The `on_init_start` callback hook was deprecated in v1.6 and will be removed in v1.8.
  rank_zero_deprecation(
/root/miniconda3/envs/sheaflearn/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:233: LightningDeprecationWarning: The `on_init_end` callback hook was deprecated in v1.6 and will be removed in v1.8.
  rank_zero_deprecation("The `on_init_end` callback hook was deprecated in v1.6 and will be removed in v1.8.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

   | Name                                | Type          | Params
------------------------

Sanity Checking: 0it [00:00, ?it/s]

/root/miniconda3/envs/sheaflearn/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [40]:
import wandb

In [41]:
from os import listdir
from os.path import isfile, join
mypath = f'/root/projects/temp_checkpoints/texas_2/addenergy_hunt2'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
test_acc_list = []
for files in onlyfiles:
    path = os.path.join(mypath,files)
    print(path)
    model_diffusion = sheaf_gradient_flow_functor.load_from_checkpoint(path)
    a = trainer.test(model_diffusion, sheaf_dm)
    test_acc_list.append(a[0]['test_acc'])
print(max(test_acc_list))
wandb.log({"max_test_acc":max(test_acc_list)})

/root/projects/temp_checkpoints/texas_2/addenergy_hunt2/memory-functoredge-vertex-edge-Jan02-epoch=39-val_loss=0.9260-val_acc=0.6780.ckpt


TypeError: sheaf_gradient_flow_functor.__init__() missing 14 required positional arguments: 'from_vertex_outgoing_edges', 'from_vertex_incoming_edges', 'from_vertex_outgoing_vertex', 'from_vertex_incoming_vertex', 'degree_list', 'NVertEnergy', 'NEdgeEnergy', 'nonlinearoutput', 'add_adjacency', 'add_stalk_mixing', 'add_channel_mixing', 'apply_spectralnorm', 'apply_symmetrize', and 'vertex_potential_type'

In [30]:
print(max(test_acc_list))


0.6486486196517944


In [ ]:
torch.diag(1.+5*torch.ones(3))

In [19]:
import torch
from torch_sparse import coalesce

In [23]:
index = torch.tensor([[1, 0, 1, 0, 2, 1],
                      [0, 1, 1, 1, 0, 0]])
value = torch.Tensor([1,2,3,4,5,6])

index, value = coalesce(index, value, m=3, n=2)

In [24]:
index

tensor([[0, 1, 1, 2],
        [1, 0, 1, 0]])

In [25]:
value

tensor([6., 7., 3., 5.])